In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from uncertainties import ufloat
import uncertainties.unumpy as unumpy
import scipy.odr as ODR

# Experimento 1

In [ ]:
datos1 = pd.DataFrame(columns=['frec', 'vi', 'vo', 'dt', 'errvo', 'nodo'])

In [ ]:

# Función para agregar datos al DataFrame
def nuevo_dato(dataframe, datos):
    nuevo_df = pd.DataFrame([datos], columns=dataframe.columns)
    return pd.concat([dataframe, nuevo_df], ignore_index=True)

# Función para borrar datos del DataFrame
def borrar_dato(dataframe, indice):

    return dataframe.drop(index=indice).reset_index(drop=True)


## Agregando datos

In [ ]:
# Columnas del dataframe:   'frec', 'vi', 'vo', 'dt', 'errvo'
datos1 = nuevo_dato(datos1, [1.0, 1.0, 1.0, 1.00, 1.00, 6.0])
datos1 = nuevo_dato(datos1, [2.0, 2.0, 2.0, 2.00, 2.00, 5.0])
datos1 = nuevo_dato(datos1, [0.0, 0.0, 0.0, 0.00, 0.00, 6.0])
datos1 = nuevo_dato(datos1, [0.0, 0.0, 0.0, 0.00, 0.00, 6.0])
datos1 = nuevo_dato(datos1, [0.0, 0.0, 0.0, 0.00, 0.00, 6.0])
datos1 = nuevo_dato(datos1, [0.0, 0.0, 0.0, 0.00, 0.00, 6.0])
datos1 = nuevo_dato(datos1, [0.0, 0.0, 0.0, 0.00, 0.00, 6.0])

In [ ]:
# Borrar todos los datos del dataframe (por si es necesario para reiniciar)
#datos1 = datos1.iloc[0:0]

In [ ]:
datos1

## Defino nuevas columnas 

In [ ]:
#def calc_err(Vi, Vo, Vi_err, Vo_err):
#  return np.sqrt((Vo_err/Vi)**2 + (Vi_err*Vo/(Vi)**2)**2)

In [ ]:
datos1['vo/vi'] = datos1['vo'] / datos1['vi']
datos1['errvi'] = datos1['vi'] * 0.05
#datos1['err_vo/vi'] = calc_err(datos1['vi'], datos1['vo'], datos1['errvi'], datos1['errvo'])
datos1['fase'] = 2 * np.pi * datos1['frec'] * datos1['dt']

In [ ]:
# Calculo err_vo/vi con ufloat
def calculate_err_vovi(row):
	if row['vi'] != 0:  # Que no divida por cero
		return round((ufloat(row['vo'], row['errvo']) / ufloat(row['vi'], row['errvi'])).std_dev, 2)
	else:
		return np.nan

datos1['err_vo/vi'] = datos1.apply(calculate_err_vovi, axis=1)

In [ ]:
datos1

## Diagrama de Bode (Amplitud)

### Ajuste

In [ ]:
x = datos1['frec']
y = datos1['vo/vi']
xerr = datos1['frec'] * 0.05
yerr = datos1['err_vo/vi']

In [ ]:
def Ajuste(B, f):
  return 2*np.pi*f*B/(np.sqrt(1+(2*np.pi*f*B)**2))

func = ODR.Model(Ajuste)
data_real = ODR.RealData(x, y, xerr, yerr)
odr = ODR.ODR(data_real, func,beta0=[0.0001])
out = odr.run()

Aj = out.beta
Aj_err = out.sd_beta

tau=ufloat(Aj[0],Aj_err[0])
print("Tau es", tau.nominal_value, "+-", tau.std_dev, "s")
print(f"Tau = {tau:.5e} s")


### Ploteo

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(x, y, c=datos1['nodo'], cmap='viridis', zorder=4, label="Datos")
ax.errorbar(x, y, xerr=xerr, yerr=yerr, fmt="none", ecolor="red", zorder=3, capsize=5)
plt.colorbar(scatter, ax=ax, label='Nodo')  # Agregar barra de color para referencia
#ax.errorbar(x, y, xerr=xerr, yerr=yerr, fmt="bo",
#            ecolor= "red", zorder=4, label="Datos",capsize=5)
ax.plot(np.logspace(1, np.log10(2*(10**6))), Ajuste(Aj[0], np.logspace(1, np.log10(2*(10**6)))), "-", color='green', zorder=3, label='Ajuste')
ax.plot(np.logspace(1, np.log10(2*(10**6))), Ajuste(1e-4, np.logspace(1, np.log10(2*(10**6)))), "-", color='teal', label="Modelo")
ax.set_xscale("log")
ax.set_yscale("log")
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(fontsize=17)
plt.xticks(fontsize='16')
plt.yticks(fontsize='16')

ax.set_xlabel("Frecuencia (Hz)", fontsize=18)
ax.set_ylabel(r"|H($\omega$)|", fontsize=18) #Sería más correcto usar H(f), los datos son de frecuencia, no frecuencia angular
ax.set_title("Diagrama de Bode (Amplitud)", fontsize=21)

plt.tight_layout()

## Diagrama de Bode (Fase)

### Ajuste

In [ ]:
x2 = datos1['frec']
y2 = datos1['fase']
xerr2 = datos1['frec'] * 0.05
yerr2 = datos1['fase'] * 0.05

In [ ]:
def Ajuste(B, f):
  return np.arctan(1/(B*2*np.pi*f))

func = ODR.Model(Ajuste)
data_real = ODR.RealData(x2, y2, xerr2, yerr2)
odr = ODR.ODR(data_real, func,beta0=[0.0001])
out = odr.run()

Aj = out.beta
Aj_err = out.sd_beta

tau=ufloat(Aj[0],Aj_err[0])
print("Tau es", Aj[0], "s")
print(f"Tau = {tau:.4e} s")

### Ploteo

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.errorbar(x2, y2, xerr = xerr2, yerr = yerr2, fmt="bo",
            ecolor= "red", zorder=4, label="Datos",capsize=5)
ax.plot(np.logspace(1, np.log10(2*(10**6))), Ajuste(Aj[0], np.logspace(1, np.log10(2*(10**6)))), "-", color='green', zorder=3, label='Ajuste')
ax.plot(np.logspace(1, np.log10(2*(10**6))), Ajuste(1e-4, np.logspace(1, np.log10(2*(10**6)))), "-", color='teal', label="Modelo")

ax.set_xscale("log")
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(fontsize=17)
plt.xticks(fontsize='16')
plt.yticks(fontsize='16')

ax.set_xlabel("Frecuencia (Hz)", fontsize=18)
ax.set_ylabel("Fase (rad)", fontsize=18)
ax.set_title("Diagrama de Bode (Fase)", fontsize=21)

plt.tight_layout()

## Diagrama de Nyquist

In [ ]:
uy = unumpy.uarray(y, yerr)
uy2 = unumpy.uarray(y2, yerr2)

ReH = uy[1:] * unumpy.cos(uy2[1:])
ImH = uy[1:] * unumpy.sin(uy2[1:])

cen_ReH = unumpy.nominal_values(ReH)
cen_ImH = unumpy.nominal_values(ImH)

err_ReH = unumpy.std_devs(ReH)
err_ImH = unumpy.std_devs(ImH)

### Ploteo

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.errorbar(cen_ReH, cen_ImH, xerr= err_ReH, yerr= err_ImH, fmt="ro", label='Datos', ecolor ='black', capsize=5)

ax.set_xlabel("Re(H)", fontsize=18)
ax.set_ylabel('Im(H)', fontsize=18)
ax.set_title("Diagrama de Nyquist", fontsize=21)

ax.grid()
ax.legend(fontsize=17)
plt.xticks(fontsize='16')
plt.yticks(fontsize='16')

plt.tight_layout()
plt.show()